TODO:
- Прочитать: https://habr.com/ru/post/349860/#Skobochnye_gruppy_gruppirovka_plyus_kvantifikatory
- extract-time можно использовать для того, чтобы указывать интервал времени (от минимального до конечного)
- Написать тестирующую функцию для подбора регулярных выражений под определенный ключевой запрос - для поиска тем и спикеров
- Для контактов - pop_contacts

# 🐅 Парсер

Парсинг начинается со страниц, перечисленных в `pages.json`. Файл содержит необходимые сведения о структуре страниц сайта: стартовая страница, селекторы страницы, отвечающие за их описания (могут быть на стартовой `start_url` или внутренних страницах `event_urk`), количество соответствующих элементов. Если для ссылки события пока нет описания в файле `events.json`, в результате парсинга создаётся соответствующий объект. Этот файл используется для генерации карточек мероприятий в HTML, публикуемый на [matyushkin.github.io/events](https://matyushkin.github.io/events/).

In [3]:
from parser import *
import re

def reload_modules():
    '''Инструмент тестирования и разработки:
    reload всех уже импортированных модулей'''
    modulenames = set(sys.modules) & set(globals())
    allmodules = [sys.modules[name] for name in modulenames]
    for module in allmodules:
        importlib.reload(module)

pages_checked = files.pages_checked()
print('Все необходимые поля имеются у следующих стартовых страниц: ')
for page in pages_checked:
    print(page)
                
for start_url in pages_checked:
    print(f"\nАнализируем стартовую страницу {start_url}")
    start_page = StartPage(start_url)
    actual = start_page.actual_events
    for i, event_url in enumerate(actual, 1):
        print(f'{i}. Обработка {event_url}')
        event = EventPage(event_url, start_page)
        del event.data['soup']
        files.events[event_url] = event.data

Все необходимые поля имеются у следующих стартовых страниц: 
https://events.yandex.ru/
https://it-events.com/

Анализируем стартовую страницу https://events.yandex.ru/
1. Обработка https://yandex.ru/promo/events/yace
'NoneType' object has no attribute 'group'
2. Обработка https://events.yandex.ru/events/ml-party-04-08-2020
3. Обработка https://events.yandex.ru/events/hardware/31-july-2020
4. Обработка https://events.webinar.ru/yandex/5706177
'NoneType' object has no attribute 'group'
5. Обработка https://cloud.yandex.ru/events/156
6. Обработка https://yandex.ru/promo/maps/meetup/retail

Анализируем стартовую страницу https://it-events.com/
1. Обработка https://it-events.com/events/18856
2. Обработка https://it-events.com/events/18920
3. Обработка https://it-events.com/events/18855
4. Обработка https://it-events.com/events/18829
5. Обработка https://it-events.com/events/18928
6. Обработка https://it-events.com/events/18911
7. Обработка https://it-events.com/events/18813
8. Обработка htt

In [4]:
with open('files/events.json', 'w', encoding='utf-8') as events_file:
    json.dump(files.events, events_file, ensure_ascii=False)

# Анализируем и дополняем полученную информацию

Предварительно, если среди мероприятий имеются мероприятия, не обрабатываемые парсером (`events_special`) или рекламируемые (`events_promo`), обрабатываем их отдельно. Для этого помечаем их в датафрейме особым образом.

In [5]:
reload_modules()
df = pd.DataFrame.from_dict(files.events, orient='index')
df['description'] = df['description'].apply(lambda x: x, 'html.parser')

In [6]:
# Сразу выкидываем мероприятия, которые не хотим видеть
df.drop(files.bad['urls'])
for theme in files.bad['themes']:
    df = df[~df.title.str.contains(theme)]

# Соортируем по дате, оставляем актуальные
df = df.sort_values(by=['date'])
df = df[df['date'] >= handlers.current_date.isoformat()]   #! добавить проверку на время


#df['type'] = df.index.map(promo_and_special())

# Вписываем теги и типы мероприятий по картам тегов и типов
df['tags'] = df.apply(lambda x: handlers.find_spec(x, 'tags'), axis=1)
df['types'] = df.apply(lambda x: handlers.find_spec(x, 'event_types'), axis=1)


# Для фильтрации мероприятий по месяцам и добавления меток "Позже" и "∞"
df['month'] = df.date.apply(langs.date_to_month)

In [7]:
# def promo_and_special():
#     event_types = 'promo', 'special'
#     d = {}
#     for event_type in event_types:
#         event_urls = list(eval(f'files.events_{event_type}.keys()'))
#         for event_url in event_urls:
#             d[event_url] = event_type
#     return d


def fill_empty_cells():
    def emptyfill(x, t):
        try:
            if math.isnan(x):
                if t == str:
                    return ''
                elif t == list:
                    return ['']
                else:
                    return set()
            else:
                return x
        except:
            return x

    for col in df:
        try:
            t = type(df[col][df[col].notnull()][0])
            df[col] = df[col].apply(lambda x: emptyfill(x, t))
        except IndexError:
            pass
        
fill_empty_cells()

https://regex101.com/

In [8]:
_ = df.description.apply(lambda x: langs.soup_to_text(BeautifulSoup(x, 'html.parser')))

# Попытаемся найти дополнительно спикеров из описания докладов
df['description'] = _.apply(lambda x: x['text'])
df['themes'] += _.apply(lambda x: x['themes'])
df['speakers'] += _.apply(lambda x: x['speakers'])

# В строках устраним лишние пробелы
df.description = df.description.apply(lambda x: " ".join(x.split()))

# Удалим упоминания стран из мест
df.location = df.location.apply(lambda x: {re.sub(',? (?:Россия|Украина|Беларусь)', '', x)})

# Соединяем онлайн-статус и локацию
df.location = df.apply(lambda x: x.location| {x.online_status}, 1).apply(lambda x: x-{'Offline', ''})

Найдем стоимость участия

In [9]:
df.price = df.price.apply(lambda x: re.sub('Стоимость участия:[\n\s]*', '', x))
df.price = df.price.apply(lambda x: re.sub('\s*(?:руб.|RUB|rub)', ' ₽', x))
df.price = df.price.apply(lambda x: re.sub('\s*(?:EURO)', ' €', x))
df.price = df.price.apply(lambda x: re.sub('\s*(?:USD)', ' $', x))
df.price = df.price.apply(lambda x: re.sub('\s*(?:UAH|грн.)', ' ₴', x))

In [10]:
df

,title,date,start_url,event_url,location,organizers,reg_url,online_status,speakers_companies,time,themes,description,speakers,price,event_type,tags,types,month
https://yandex.ru/promo/maps/meetup/retail,Карты и Навигатор: рекламные возможности для р...,2020-07-28,https://events.yandex.ru/,https://yandex.ru/promo/maps/meetup/retail,{},[Яндекс],https://yandex.ru/promo/maps/meetup/retail#form,Offline,"[Яндекс, iConText]",10:00,"[, Покупательские привычки пользователей Геосе...",,"[Кирилл Воробьев, Александр Локтев, Илья Цинке...",0,,{Geo},{Митап},Июль
https://it-events.com/events/18940,Интенсивный курс React Essential,2020-07-28,https://it-events.com/,https://it-events.com/events/18940,{Online},[],https://bit.ly/2CTEFp6,Online,[],19:00,[],Подробная программа. Регистрация на курс на са...,[],уточняйте у организатора,Курс,{Frontend},"{Курс, Конференция}",Июль
https://it-events.com/events/18937,IT-стартапы и банки: как доказать ценность и п...,2020-07-28,https://it-events.com/,https://it-events.com/events/18937,{Online},[],https://edu.iidf.ru/?arrangement=5f5b17d0,Online,[],17:00,[],28 июля на онлайн-конференции «Акселератора ФР...,[],0,Конференция,{},{Конференция},Июль
https://it-events.com/events/18108,Как сократить расходы на ИТ с помощью монитори...,2020-07-28,https://it-events.com/,https://it-events.com/events/18108,{Online},[],https://db-service.ru/webinar_kak-sokratit-ras...,Online,[],12:00,[],Руководите или отвечаете за работу ИТ-подразде...,"[, Виталий Савенков]",0,Вебинар,{},{Конференция},Июль
https://it-events.com/events/18941,Создание цифрового двойника на примере цеховой...,2020-07-28,https://it-events.com/,https://it-events.com/events/18941,{Online},[],https://winnum.timepad.ru/event/1382648/,Online,[],11:00,[],Компания Winnum - ведущий лидер в области пере...,[],0,Вебинар,{},{Конференция},Июль
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://it-events.com/events/14370,Интеллектуальный турнир «IT Brain Battle Mosco...,2020-11-21,https://it-events.com/,https://it-events.com/events/14370,{Москва},[],mailto:maria@it-events.com,Offline,[],10:00,[],21 ноября 2020 года в рамках Кубка «IT-Challen...,[],60 000 - 65 000 ₽,,{},{Конференция},Ноябрь
https://it-events.com/events/11470,Автоматизация управления по целям и KPI в сред...,2020-11-26,https://it-events.com/,https://it-events.com/events/11470,{Online},[],https://clck.ru/PLmH6,Online,[],14:00,[],,"[, Александр Белов]",0,Вебинар,{Auto},{Конференция},Ноябрь
https://it-events.com/events/17291,Golang Piter 2020,2020-11-27,https://it-events.com/,https://it-events.com/events/17291,"{Online, Санкт-Петербург}",[],mailto:irina@it-events.com,Online,[],09:00,[],VISA FREE FOR 53 COUNTRIES Международная техни...,[],8 000 - 14 000 ₽,Конференция,{Go},{Конференция},Ноябрь
https://it-events.com/events/17012,Турнир по кикеру «IT's KICKER St.Petersburg 2020»,2020-12-05,https://it-events.com/,https://it-events.com/events/17012,{Санкт-Петербург},[],mailto:maria@it-events.com,Offline,[],10:00,[],"5 декабря 2020 года в рамках Кубка ""IT-Challen...",[],60 000 - 65 000 ₽,,{},"{IT-спорт, Конференция}",Декабрь


# Собираем HTML-страницу и деплоим проект 💃

Сформируем блок вступления. Определим имеющиеся страницы сбора, общее количество выводимых мероприятий, даты.

In [11]:
reload_modules()
import pymorphy2
import datetime
morph = pymorphy2.MorphAnalyzer()
m = morph.parse('событие')[0]

n = df.shape[0]
github_events_url = "https://github.com/matyushkin/events"

intro = f"""<section><p>На этой странице приведен список IT-мероприятий, \
собранных <a href="{github_events_url}">специально разработанным парсером</a>. \
Парсер считывает информацию со страниц мероприятий \
{langs.string_of_page_checked_urls(pages_checked)}. \
Сегодня он собрал {n} {m.make_agree_with_number(n).word} \
{langs.date_interval_string(df.date[0], df.date[n-1])}. \
Чтобы предложить свое мероприятие, заполните <a href=\"https://forms.gle/wHsFntdTvr33b4fCA\">форму</a>.
Если вы хотите добавить свой ресурс, \
пишите в телеграм <a href=\"https://t.me/bythi\">@bythi</a> \
или на почту leva.matyushkin@gmail.com.</p>\
<p>Чтобы найти интересующее мероприятие, \
воспользуйтесь панелью фильтров.</p>\
</section>"""

print(intro)

<section><p>На этой странице приведен список IT-мероприятий, собранных <a href="https://github.com/matyushkin/events">специально разработанным парсером</a>. Парсер считывает информацию со страниц мероприятий <a href="https://events.yandex.ru/">Яндекса</a> и <a href="https://it-events.com/">IT-Events</a>. Сегодня он собрал 91 событие с 28 июля по 19 декабря 2020 года. Чтобы предложить свое мероприятие, заполните <a href="https://forms.gle/wHsFntdTvr33b4fCA">форму</a>.
Если вы хотите добавить свой ресурс, пишите в телеграм <a href="https://t.me/bythi">@bythi</a> или на почту leva.matyushkin@gmail.com.</p><p>Чтобы найти интересующее мероприятие, воспользуйтесь панелью фильтров.</p></section>


Для сборки страницы используем BeautifulSoup. Страницу собираем, объединяя soup-объекты в один.

In [12]:
from bs4 import BeautifulSoup, Comment
from itertools import zip_longest
import copy

with open("files/event_card_template.html") as template:
    template = BeautifulSoup(template.read(), 'html.parser')


def add_info(template, data):
    s = copy.copy(template)
    
    # Информационный блок
    data_block = s.find("div", {"class":"event"})
    companies = [*data.organizers, *data.speakers_companies]
    companies = " ".join(list(set(companies))).strip()
    data_block['data-companies'] = companies
    
    data_block['data-month'] = data.month
    data_block['data-price'] = data.price
    data_block['data-online'] = data.online_status
    if data.tags:
        data_block['data-tags'] = ", ".join(data.tags)
    else:
        data_block['data-tags'] = ""
    if data.types:
        data_block['data-types'] = ", ".join(data.types)
    else:
        data_block['data-types'] = ""
    if data.location:
        data_block['data-loc'] = ", ".join(data.location)
    else:
        data_block['data-loc'] = ""
#     if data.type == 'promo':
#         data_block['class'].append('promo')
    
    # Обработка заголовка
    header = s.find("h2")
    header.string = data.title
    
    # Блок с описанием места и времени
    time_and_space = s.find("p", {"class":"time_and_space"})
    s.time.string = langs.make_datetime_string(data.date, data.time)
    s.time.wrap(s.new_tag('a', attrs={'href':data.event_url}))
    if data.reg_url:
        time_and_space.append(', ')
        reg_url = s.new_tag('a', attrs={'href':data.reg_url})
        reg_url.string = 'регистрация'
        time_and_space.append(reg_url) 
    time_and_space.append('.')
    
    # Блок с темами и докладчиками
    themes_and_speakers = s.find("p", {"class":"themes_and_speakers"})
    
    if data.themes != ['']:
        for theme in data.themes:
            if theme:
                line = s.new_tag('li')
                line.string = theme
                s.ul.append(line)
    if data.speakers != ['']:
        if len(data.speakers) == 1:
            themes_and_speakers.append(f'👤 {data.speakers[0]}.')
        else:
            speakers = [x for x in data.speakers if x]
            speakers_str = ', '.join(speakers)
            themes_and_speakers.append(f'👥 {speakers_str}.')
    if data.themes == [''] and data.speakers != ['']:
        s.ul.extract()
    if data.themes == [''] and data.speakers == ['']:
        s.find("p", {"class":"themes_and_speakers"}).extract()
    
    # Блок с описанием мероприятия
    description = s.find("p", {"class":"description"})
    description.string = data.description
    
    # Удаляем комментарии
    for element in s(text = lambda text: isinstance(text, Comment)):
        element.extract()

    return s

df['soup'] = df.apply(lambda x: add_info(template, x), axis=1)
df['html'] = df['soup'].apply(lambda x: str(x))
total = ''.join(df['html'])

total += '''<div class="filter__msg" hidden>
<p>🕵 К сожалению, для выбранных фильтров пока нет ни одного подходящего мероприятия.
Попробуйте выбрать другие или вернитесь завтра – парсер поищет новые.</p></div>'''

path_to_html_template = "../mgio/11ty/_includes/events.njk"
with open(path_to_html_template, 'w', encoding='utf-8') as html_template:
    html_template.write(f'{total}')

Используем тот же подход для сборки меню фильтров. 

In [18]:
import langs

all_tags = set(itertools.chain.from_iterable(df.tags))
all_types = set(itertools.chain.from_iterable(df.types))
all_locs = set(itertools.chain.from_iterable(df.location)) - {''}
all_months = langs.month_names_for_time_filters()[1:] + ['Позже', '∞']
#all_types = set(itertools.chain.from_iterable(df.type))

with open("files/event_filter_panel_template.html") as filter_template:
    filter_soup = BeautifulSoup(filter_template.read(), 'html.parser') 


def filter_item_append(name, value):
    s = filter_soup.find('div', {'class':f'filter__container--{name}'}).div
    t = filter_soup.new_tag('div')
    t['class'] = 'filter__item filter__button'
    t.string = value
    s.append(t)
    

for tag in all_tags:
    filter_item_append('tag', tag)

for type_ in all_types:
    filter_item_append('type', type_)

for month in all_months:
    filter_item_append('month', month)

for loc in all_locs:
    filter_item_append('loc', loc)

    
with open("../mgio/11ty/_includes/events_filter.njk", 'w', encoding='utf-8') as html_template:
    html_template.write(intro + str(filter_soup))


!cp -rp mgio.11ty.events/* ../mgio/11ty/events
!cd ../mgio/11ty/; npx eleventy --passthroughall --output=../../matyushkin.github.io
!cd ../matyushkin.github.io/; rm -rf 404
!cd ../matyushkin.github.io/; rm -rf README


# посмотрим, что получилось в браузере
import webbrowser
url = "../matyushkin.github.io/events/index.html"
webbrowser.open_new_tab(url)

Writing ../../matyushkin.github.io/404/index.html from ./404.html.
Writing ../../matyushkin.github.io/README/index.html from ./README.md.
Writing ../../matyushkin.github.io/index.html from ./index.html.
Writing ../../matyushkin.github.io/cv/index.html from ./cv/index.html.
Writing ../../matyushkin.github.io/donate/index.html from ./donate/index.html.
Writing ../../matyushkin.github.io/posts/index.html from ./posts/index.html.
Writing ../../matyushkin.github.io/events/index.html from ./events/index.html.
Writing ../../matyushkin.github.io/links/index.html from ./links/index.html.
Writing ../../matyushkin.github.io/spb/index.html from ./spb/index.html.
Writing ../../matyushkin.github.io/texts/index.html from ./texts/index.html.
Copied 18 files / Wrote 10 files in 0.23 seconds (23.0ms each, v0.11.0)


True

In [19]:
# List of events is updated

!cd ../matyushkin.github.io/; git add . ; git commit -m "JS for Price btn is added "; git push origin master

[master f7670b7] JS for Price btn is added
 2 files changed, 34 insertions(+), 30 deletions(-)
 rewrite events/index.html (86%)
Counting objects: 6, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 1.88 KiB | 148.00 KiB/s, done.
Total 6 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To github.com:matyushkin/matyushkin.github.io.git
   488e883..f7670b7  master -> master


In [ ]:
# посмотрим, что выгрузилось на сайт
import webbrowser
url = "https://matyushkin.github.io/events/"
webbrowser.open_new_tab(url)

In [ ]:
df

In [ ]:
df1